**Sample ID**: CM129_base


**Query**: Turn on the living room TV if the toys have been put back in their totes in Jimmy’s room.


**DB Type**: Base Case


**Case Description**: Initially, the Jimmy Room Cam exists and monitors whether the toys in Jimmy’s room are placed back in their totes. The Living Room TV is present in the system and is currently off.


**Global/Context Variables:**

**APIs:**
- home_assistant
- sdm

# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

VERSION = "0.1.4"  # Pass the version of the API
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Cleanup
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Auth
auth.authenticate_user()
drive_service = build('drive', 'v3')

def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(output_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None
try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    for file in results.get('files', []):
        if file['name'].lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file['id']
            print(f"Found: {file['name']} (ID: {apis_file_id})")
            break
except Exception as e:
    print(f"Error listing files: {e}")

if not apis_file_id:
    sys.exit(f"❌ APIs zip V{VERSION} not found.")

print(f"Downloading APIs zip {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH)

print(f"Extracting {ZIP_PATH}...")
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    for member in zip_ref.namelist():
        if any(member.startswith(p) for p in ITEMS_TO_EXTRACT):
            zip_ref.extract(member, CONTENT_DIR)

os.remove(ZIP_PATH)

if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)

for p in [APIS_DIR, DBS_DIR, SCRIPTS_DIR]:
    print(f"{'✅' if os.path.exists(p) else '❌'} {p}")

# PWS: Copy pws_image_data folder in /content/pws_image_data
print(f"\nDownloading all files from pws_images_data Drive folder.")
IMAGES_ZIP_ID = '1MlgE9LnFA221ARh9l4nbF1yGc215KDN_'
IMAGES_ZIP_PATH = os.path.join(CONTENT_DIR, 'pws_images_data.zip')
download_drive_file(drive_service, IMAGES_ZIP_ID, IMAGES_ZIP_PATH, file_name='pws_images_data.zip')

print(f"Extracting images from {IMAGES_ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(IMAGES_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(CONTENT_DIR)

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {IMAGES_ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")

if os.path.exists(IMAGES_ZIP_PATH):
    os.remove(IMAGES_ZIP_PATH)

Searching for APIs zip file with version 0.1.4 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found: APIs_V0.1.4.zip (ID: 1TnAaWGfVrMxWTilyhy46-Aue_bh0XkNk)
Download progress: 100%
Extracting /content/APIs_V0.1.4.zip...
✅ /content/APIs
✅ /content/DBs
✅ /content/Scripts

Download progress: 78%
Download progress: 100%
Extracting images from /content/pws_images_data.zip to /content...


## Install Dependencies and Clone Repositories

In [ ]:
!pip install uv
!uv pip install -r /content/APIs/requirements.txt
!uv pip install jsonpath_ng

Using Python 3.12.12 environment at: /usr
Audited 50 packages in 100ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 115ms


## Import APIs and initiate DBs

In [ ]:
# proto_ignore

scenario_db = {'environment': {'home_assistant': {'devices': {'LIGHT_001': {'attributes': {'brightness': 1.0,
                                                                             'company': 'Crompton',
                                                                             'model': '4Ft.22W',
                                                                             'serial_number': 'C145098',
                                                                             'state': 'Off',
                                                                             'supports_color': 'No'},
                                                              'name': 'Living '
                                                                      'Room '
                                                                      'Light 1',
                                                              'type': 'Light'},
                                                'LIGHT_002': {'attributes': {'brightness': 1.0,
                                                                             'company': 'Crompton',
                                                                             'model': '4Ft.22W',
                                                                             'serial_number': 'C567146',
                                                                             'state': 'Off',
                                                                             'supports_color': 'No'},
                                                              'name': 'Living '
                                                                      'Room '
                                                                      'Light 2',
                                                              'type': 'Light'},
                                                'TV_001': {'attributes': {'brightness': 50,
                                                                          'company': 'Samsung',
                                                                          'is_muted': 'No',
                                                                          'model': '65-inch '
                                                                                   'QLED '
                                                                                   'Smart '
                                                                                   'TV',
                                                                          'serial_number': 'SAM-65Q900',
                                                                          'state': 'Off',
                                                                          'volume': 20},
                                                           'name': 'Living '
                                                                   'Room TV',
                                                           'type': 'tv'},
                                                'WIFI_001': {'attributes': {'company': 'TP-Link',
                                                                            'max_speed': 'Up '
                                                                                         'to '
                                                                                         '6 '
                                                                                         'Gbps',
                                                                            'model': 'ARCHER '
                                                                                     'C6 '
                                                                                     '(AC1200)',
                                                                            'security_protocol': 'WPA3-Personal',
                                                                            'serial_number': '255156382956',
                                                                            'state': 'On',
                                                                            'wifi_standard': 'Wi-Fi '
                                                                                             '6 '
                                                                                             '(802.11ax)'},
                                                             'name': 'TP-Link '
                                                                     'ARCHER '
                                                                     'C6',
                                                             'type': 'Wifi'}}},
                 'sdm': {'devices': {'CAM_001': {'attributes': {'company': 'Google',
                                                                'model': 'Google '
                                                                         'Nest '
                                                                         'Cam '
                                                                         'Indoor',
                                                                'parent': 'enterprises/home-system-546237/structures/STRCT2/rooms/JIMMY_ROOM',
                                                                'serial_number': 'GN-I-987633',
                                                                'state': 'On'},
                                                 'name': 'Jimmy Room Cam',
                                                 'type': 'Camera'},
                                     'CAM_002': {'attributes': {'company': 'Google',
                                                                'model': 'Google '
                                                                         'Nest '
                                                                         'Cam '
                                                                         'Indoor',
                                                                'parent': 'enterprises/home-system-546237/structures/STRCT2/rooms/LIVING_ROOM',
                                                                'serial_number': 'GN-I-07062500',
                                                                'state': 'On'},
                                                 'name': 'Living Room',
                                                 'type': 'Camera'},
                                     'CAM_003': {'attributes': {'company': 'Google',
                                                                'model': 'Google '
                                                                         'Nest '
                                                                         'Cam '
                                                                         'IQ '
                                                                         'Outdoor',
                                                                'parent': 'enterprises/home-system-546237/structures/STRCT1/rooms/OUT1',
                                                                'serial_number': 'GNC-IQ-O-19000',
                                                                'state': 'On'},
                                                 'name': 'Backyard Cam',
                                                 'type': 'Camera'}},
                         'structures': [{'name': 'enterprises/home-system-546237/structures/STRCT1',
                                         'traits': {'sdm.structures.traits.Info': {'customName': 'House '
                                                                                                 '- '
                                                                                                 'Outdoors'}}},
                                        {'name': 'enterprises/home-system-546237/structures/STRCT2',
                                         'traits': {'sdm.structures.traits.Info': {'customName': 'House '
                                                                                                 '- '
                                                                                                 'Indoors'}}}]}},
 'project_id': 'home-system-546237'}

image_map = {'cameras': {'CAM_001': [{'expected': 'Yes',
                          'image_path': './pws_image_data/05-16-25 '
                                        '133421.jpg'}],
             'CAM_002': [{'expected': 'No',
                          'image_path': './pws_image_data/05-16-25 '
                                        '115316.jpg'}],
             'CAM_003': [{'expected': 'No',
                          'image_path': './pws_image_data/05-22-2025 '
                                        '151727.jpg'}]},
 'events': []}


import json

with open("/content/scenario_db.json", "w") as f:
    json.dump(scenario_db, f)

with open("/content/image_map.json", "w") as f:
    json.dump(image_map, f)


import json
import home_assistant
import sdm

# Initialize DBs
home_assistant.SimulationEngine.db.load_state(f"/content/scenario_db.json")
sdm.SimulationEngine.db.load_state(f"/content/scenario_db.json")

# Print the DBs
print(f"Home Assistant DB: {home_assistant.SimulationEngine.db.DB}")
print(f"SDM DB: {sdm.SimulationEngine.db.DB}")

# Load image map for this task
with open("/content/image_map.json", "r") as f:
    image_map = json.load(f)

# Print the Image Map
print(f"Image Map: {image_map}")

# Reset states
sdm.devices.commands.reset_served_images()
sdm.SimulationEngine.db.reset_state_env()

# Initialize image map
sdm.SimulationEngine.db.update_state_dict(image_map)

# Set events
sdm.SimulationEngine.events.set_cameras_events(image_map, home_assistant.SimulationEngine.db.DB.get("project_id", ""))

Home Assistant DB: {'environment': {'home_assistant': {'devices': {'LIGHT_001': {'attributes': {'brightness': 1.0, 'company': 'Crompton', 'model': '4Ft.22W', 'serial_number': 'C145098', 'state': 'Off', 'supports_color': 'No'}, 'name': 'Living Room Light 1', 'type': 'Light'}, 'LIGHT_002': {'attributes': {'brightness': 1.0, 'company': 'Crompton', 'model': '4Ft.22W', 'serial_number': 'C567146', 'state': 'Off', 'supports_color': 'No'}, 'name': 'Living Room Light 2', 'type': 'Light'}, 'TV_001': {'attributes': {'brightness': 50, 'company': 'Samsung', 'is_muted': 'No', 'model': '65-inch QLED Smart TV', 'serial_number': 'SAM-65Q900', 'state': 'Off', 'volume': 20}, 'name': 'Living Room TV', 'type': 'tv'}, 'WIFI_001': {'attributes': {'company': 'TP-Link', 'max_speed': 'Up to 6 Gbps', 'model': 'ARCHER C6 (AC1200)', 'security_protocol': 'WPA3-Personal', 'serial_number': '255156382956', 'state': 'On', 'wifi_standard': 'Wi-Fi 6 (802.11ax)'}, 'name': 'TP-Link ARCHER C6', 'type': 'Wifi'}}}, 'sdm': {'d

# Initial Assertion
1. Assert that the "Living Room TV" is turned off.

In [ ]:
import home_assistant
import sdm
from Scripts.assertions_utils import *

# Local variables
living_room_tv_name = "Living Room TV"

# 1. Assert that the living room tv is off
try:
  living_room_tv_id = home_assistant.get_id_by_name(living_room_tv_name)
  tv_state = home_assistant.get_state(living_room_tv_id)
except Exception as e:
  tv_state = {}

assert compare_strings(tv_state.get("state", ""), "Off"), "TV in living room is turned on, expected it to be Off."

# Action
* Locate Jimmy Room camera.
* Access current footage "05-16-25 133421"
* The camera shows two closed totes on the floor, and there are no longer toys scattered on the area rug.
* Turn on "Living Room TV"

In [ ]:
import home_assistant
import sdm
import base64
#from IPython.display import Image, display

In [ ]:


def display_base64_image(image_base64):
    """Helper function to display a base 64 image"""
    base64_string = image_base64.split(':')[1].strip()

    try:
        missing_padding = len(base64_string) % 4
        if missing_padding:
            base64_string += '=' * (4 - missing_padding)
        image_data = base64.b64decode(base64_string)
    except Exception as e:
        print(f"Error decoding base64 string: {e}")
        image_data = None

    if image_data:
        display(Image(data=image_data))

# 1. List all relevant devices
# List all the televisions
tvs = home_assistant.list_devices("tv").get("entities", [])
print(f"All available televisions: {tvs}")

# List all sdm devices
sdm_devices = sdm.list_devices()
print(f"All available SDM devices: {sdm_devices}")

All available televisions: [{'TV_001': {'attributes': {'brightness': 50, 'company': 'Samsung', 'is_muted': 'No', 'model': '65-inch QLED Smart TV', 'serial_number': 'SAM-65Q900', 'state': 'Off', 'volume': 20}, 'name': 'Living Room TV', 'type': 'tv'}}]
All available SDM devices: {'devices': [{'name': 'enterprises/home-system-546237/devices/CAM_001', 'type': 'sdm.devices.types.CAMERA', 'traits': {'sdm.devices.traits.CameraEventImage': {}, 'sdm.devices.traits.CameraImage': {'maxImageResolution': {'width': 1280, 'height': 960}}, 'sdm.devices.traits.CameraLiveStream': {'maxVideoResolution': {'width': 640, 'height': 480}, 'videoCodecs': ['H264'], 'audioCodecs': ['AAC'], 'supportedProtocols': ['RTSP', 'WEB_RTC']}, 'sdm.devices.traits.CameraMotion': {}, 'sdm.devices.traits.CameraPerson': {}, 'sdm.devices.traits.CameraSound': {}, 'sdm.devices.traits.Info': {'customName': 'Jimmy Room Cam'}}, 'project_id': 'home-system-546237', 'parentRelations': [{'parent': 'enterprises/home-system-546237/structu

In [ ]:
# Set required ids
# Assuming "Living Room TV" is the tv present in Living Room
living_room_tv_id = "TV_001"
# Assuming "Jimmy Room Cam" is the camera present in Jimmy's room
jimmy_room_cam_id = "CAM_001"
# From SDM devices output
project_id = "home-system-546237"

# 2. Load Jimmy's room image
# Start RTSP stream
generate_stream_command = {"command": "sdm.devices.commands.generate_rtsp_stream",
                           "params": {}
                           }
generate_stream_output = sdm.execute_command(device_id=jimmy_room_cam_id, project_id=project_id, command_request=generate_stream_command)

# Inspect stream output
print(f"Generated stream output: {generate_stream_output}")

Generated stream output: {'results': {'streamUrls': {'rtspUrl': 'rtsps://sdmvideostream.com/Zy4wLnN0cmVhbWluZ1Rva2VuLi9wd3NfaW1hZ2VfZGF0YS8wNS0xNi0yNSAxMzM0MjEuanBn?auth=g.0.streamingToken'}, 'streamExtensionToken': 'nEiJEGFyQuiy-pCFMGbqBw', 'streamToken': 'g.0.streamingToken', 'expiresAt': '2025-10-21T10:18:00.265666'}}


In [ ]:
# Get image from stream
generate_image_command = {"command": "sdm.devices.commands.generate_image_from_rtsp_stream",
                          "params": {
                              "rtsp_url": generate_stream_output.get("results", {}).get("streamUrls", "").get("rtspUrl", "")
                              }
                          }
image_1 = sdm.execute_command(device_id=jimmy_room_cam_id, project_id=project_id, command_request=generate_image_command)

# Stop RTSP stream
stop_stream_command = {"command": "sdm.devices.commands.stop_rtsp_stream",
                       "params": {
                           "stream_extension_token": generate_stream_output.get("results", {}).get("streamExtensionToken", "")
                           }
                       }
stop_stream_output = sdm.execute_command(device_id=jimmy_room_cam_id, project_id=project_id, command_request=stop_stream_command)

# THIS IS JUST FOR DEBUGGING
#display_base64_image(image_1)

In [ ]:
# Mock reply from Gemini
print("The floor is noticeably more organized, with most toys neatly stored in \
two large bins—one transparent and one with a white lid—placed on the space-themed mat. \
Previously scattered items like LEGO pieces have been cleaned up, leaving only \
a few objects like a red bucket, a green bag, and some pencils near the nightstand.")


# 3. Turn on the living room TV
home_assistant.toggle_device(living_room_tv_id, 'On')

The floor is noticeably more organized, with most toys neatly stored in two large bins—one transparent and one with a white lid—placed on the space-themed mat. Previously scattered items like LEGO pieces have been cleaned up, leaving only a few objects like a red bucket, a green bag, and some pencils near the nightstand.


{'status': 'SUCCESS'}

# Final Assertion
1. Assert that the "Living Room TV" is turned on.
2. Assert that all the relevant images are utilized.

In [ ]:
import home_assistant
import sdm
from Scripts.assertions_utils import *

# Local variables
living_room_tv_name = "Living Room TV"

# Helper functions
def are_all_images_used(expected_paths: list[str], served_type: str) -> bool:
    # Get served images
    served_image_paths = sdm.devices.commands.get_served_images()
    if compare_strings(served_type , "event"):
        image_paths = served_image_paths.get("EVENT_IMAGES", [])
    elif compare_strings(served_type , "stream"):
        image_paths = served_image_paths.get("STREAM_IMAGES", [])
    else:
        return False

    output = False
    expected_paths = list(set(expected_paths))
    image_paths = list(set(image_paths))

    if expected_paths and image_paths:
        output = compare_is_list_subset(expected_paths, image_paths, list_comparison_function="all")
    elif not expected_paths and not image_paths:
        output = True
    return output

# 1. Assert that living room tv is on
try:
  living_room_tv_id = home_assistant.get_id_by_name(living_room_tv_name)
  tv_state = home_assistant.get_state(living_room_tv_id)
except Exception as e:
  tv_state = {}

assert compare_strings(tv_state.get("state", ""), "On"), "TV in living room is turned Off, expected it to be On."

# 2. Assert that all the relevant images are utilized.
# Collect expected image paths from the image map
image_map = sdm.load_image_map()
all_expected_paths = []
camera_image_entries = image_map.get("cameras", {})
event_image_entries = image_map.get("events", [])

for camera_id, image_entries in camera_image_entries.items():
    for entry in image_entries:
        if compare_strings(entry.get("expected", ""), "Yes"):
            all_expected_paths.append(entry.get("image_path"))

expected_event_paths = []
for event in event_image_entries:
    expected_event_paths.append(event.get("image_path"))

expected_stream_paths = [path for path in all_expected_paths if path not in expected_event_paths]

# Assert that all relevant images are used
assert are_all_images_used(expected_event_paths, "event"), "Mismatch: Not all expected event images were used."
assert are_all_images_used(expected_stream_paths, "stream"), "Mismatch: Not all expected stream images were used."